In [ ]:
!pip install -Uq tensorflow_hub  pydub

In [ ]:
!pip install -Uq tensorflow-io

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 21.4 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the YAMNET model
model = hub.load('https://tfhub.dev/google/yamnet/1') #yamnet is the model we are using, as for face we used facenet but for audio we are using yamnet

# Load an audio file
audio, sample_rate = tf.audio.decode_wav(tf.io.read_file('audio/h1.wav'))
audio = tf.squeeze(audio, axis=-1)

# Generate embeddings
scores, embeddings, log_mel_spectrogram = model(audio)

print(f"Audio embedding shape: {embeddings.shape}")

NotFoundError: {{function_node __wrapped__ReadFile_device_/job:localhost/replica:0/task:0/device:CPU:0}} audio/h1.wav; No such file or directory [Op:ReadFile]

In [ ]:
# Load an audio file
audio, sample_rate = tf.audio.decode_wav(tf.io.read_file('audio/a2.wav'))
audio = tf.squeeze(audio, axis=-1)

scores, embeddings, log_mel_spectrogram = model(audio)

print(f"Audio embedding shape: {embeddings.shape}")#27,18

In [ ]:
log_mel_spectrogram

In [ ]:
import os
import numpy as np

voices = []
labels = []
for i in os.listdir("./audio/"):
    if '.wav' in i:
        name = i.split(".")[0]


        audio, sample_rate = tf.audio.decode_wav(tf.io.read_file(f'./audio/{i}'))
        audio = tf.squeeze(audio, axis=-1)

        scores, embeddings, log_mel_spectrogram = model(audio)

        voices.append(np.array(embeddings[:5,:]).ravel())
        labels.append(name)

        print(f"Audio embedding shape: {embeddings.shape} new shape {embeddings[:5,:].shape} type {type(np.array(embeddings[:18,:]))}")#27,18
        print(i)

In [ ]:
voices

In [ ]:
#Imports a PyMilvus package:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

#Connect to the Milvus
connections.connect("default", host="localhost", port="19530")
# Define the collection name
collection_name = "audio"

# Delete old collection if it exists
if utility.has_collection(collection_name):
    Collection(collection_name).drop()


#Creates a collection:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="words", dtype=DataType.VARCHAR, max_length=50),
    FieldSchema(name="person_name", dtype=DataType.VARCHAR, max_length=50),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=5120)
]
schema = CollectionSchema(fields, "Simple Demo for audio similar search")
audio = Collection("audio", schema)


In [ ]:
# Builds indexes on the entities:

index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}

audio.create_index("embeddings", index)

In [ ]:
labels

In [ ]:
voices[0].shape

In [ ]:
#Insert data in collection
data = [
    [1,2,3,4,5,6],  # field pk
    labels,  # field words
    ["Auranzaib","Auranzaib","Hasnant","Tasmeer","Hasnant","tasmeer"],
    voices,  # field embeddings
]

In [ ]:
audio.insert(data)
audio.flush()
audio.load()

In [ ]:
search_params = {"metric_type": "L2"}

In [2]:
results = audio.search(
	data=[voices[0]],
	anns_field="embeddings",
	param=search_params,
	limit=4,
	expr=None,
	# set the names of the fields you want to retrieve from the search result.
	output_fields=['words','person_name'],
	consistency_level="Strong"
)

NameError: name 'audio' is not defined

In [3]:
for i in range(0,len(results[0])):
    name = results[0][i].entity.get('words')
    pname = results[0][i].entity.get('person_name')
    print(pname)
#     try:
#         display(Image.open('./images/'+name+'.jpg'))
#     except:
#         display(Image.open('./images/'+name+'.png'))

NameError: name 'results' is not defined

In [ ]:
labels